<a href="https://colab.research.google.com/github/ma2sevich222/raw_stock_data_researches/blob/main/dynamical%20barycenter%20average%20implement%20for%20search%20patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fastdtw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
from scipy.spatial.distance import euclidean
from scipy import stats
from fastdtw import fastdtw

In [3]:
import plotly.graph_objects as go
import io
from google.colab import files
import pandas as pd
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm


In [ ]:
'''import pandas as pd
import numpy as np

def calculate_macd(df: pd.DataFrame, minutes_ema12: int, minutes_ema26: int, minutes_signal: int) -> pd.DataFrame:
    # Calculate the EMA12 and EMA26
    df['ema12'] = df['Close'].ewm(span=minutes_ema12, adjust=False).mean()
    df['ema26'] = df['Close'].ewm(span=minutes_ema26, adjust=False).mean()

    # Calculate the MACD line and signal line
    df['macd'] = df['ema12'] - df['ema26']
    df['signal'] = df['macd'].ewm(span=minutes_signal, adjust=False).mean()

    # Calculate the histogram
    df['histogram'] = df['macd'] - df['signal']

    return df




def calculate_rsi(df: pd.DataFrame, period: int) -> pd.DataFrame:
    diff = df['Close'].diff()
    gain = diff.where(diff > 0, 0)
    loss = diff.where(diff < 0, 0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['rsi'] = rsi
    return df


def add_momentum(ohlc, period):
    # Calculate the momentum indicator
    momentum = ohlc['Close'].diff(period)
    momentum = (momentum / ohlc['Close'].shift(period)) * 100
    
    # Replace the first NaN value with 0
    momentum.iloc[:period] = 0
    
    # Add the momentum indicator to the original dataframe
    ohlc['momentum'] = momentum
    return ohlc'''

In [ ]:
'''def add_MACD(df):
    df["Datetime"] = pd.to_datetime(df["Datetime"])
    df.set_index("Datetime", inplace=True)
    df_VWAP = None
    itr = 0
    for idx, day in df.groupby(df.index.date):
        
        day = calculate_macd(day,2,4,3)
        #day = calculate_rsi(day,60)
        #day = add_momentum(day,10)
 
        if itr == 0:
            df_VWAP = day
            itr += 1
        else:
            df_VWAP = df_VWAP.append(day, ignore_index=True)
 
    df_VWAP["Datetime"] = df.index.values
    df = df.reset_index()

    return df_VWAP'''

In [22]:
def normalize_columns(X, min_val=0, max_val=1):
    X_norm = np.zeros_like(X)
    for i in range(X.shape[1]):
        col = X[:, i]
        col_min = col.min()
        col_max = col.max()
        if col_max == col_min:
            col_norm = np.zeros_like(col)
        else:
            col_norm = (col - col_min) / (col_max - col_min)
            col_norm = min_val + (max_val - min_val) * col_norm
        X_norm[:, i] = col_norm
    return X_norm

def calculate_differences(X):
    differences = np.zeros_like(X)
    for i in range(X.shape[1]):
        differences[1:, i] = X[1:, i] - X[:-1, i]
    return differences



import numpy as np
from sklearn.decomposition import NMF

def nmf_decompose(array, n_components):
  model = NMF(n_components=n_components)
  W = model.fit_transform(array)
  H = model.components_
  return W, H

In [5]:
'''def add_VWAP_bands(df):
    
    df["Datetime"] = pd.to_datetime(df["Datetime"])
    df.set_index("Datetime", inplace=True)
    df_VWAP = None
    itr = 0
    for idx, day in df.groupby(df.index.date):
        
        day = calculate_macd(day,2,4,3)
        day["TP"] = (day["Open"] + day["High"] + day["Low"] + day["Close"]) / 4
        day["VTP"] = day["TP"] * day["Volume"]
        day["cumVTP"] = day["VTP"].cumsum()
        day["cumV"] = day["Volume"].cumsum()
        day["WVAP"] = day["cumVTP"] / day["cumV"]
        day["Var"] = (((day["TP"] - day["WVAP"]) ** 2) * day["Volume"]).cumsum()
        day["Off"] = (day["Var"] / day["cumV"]) ** 0.5
        day["+band"] = day["WVAP"] + day["Off"]
        day["-band"] = day["WVAP"] - day["Off"]

        if itr == 0:
            df_VWAP = day
            itr += 1
        else:
            df_VWAP = df_VWAP.append(day, ignore_index=True)
    del (
        df_VWAP["TP"],
        df_VWAP["VTP"],
        df_VWAP["cumVTP"],
        df_VWAP["cumV"],
        df_VWAP["Var"],
        df_VWAP["Off"],
    )
    df_VWAP["Datetime"] = df.index.values
    df = df.reset_index()

    return df_VWAP'''


def get_patterns(df, p_size):
    
    state = 0
    i = 0
    u = 0
    d = 0

    df["Datetime"] = pd.to_datetime(df["Datetime"])
    df.set_index("Datetime", inplace=True)
    sell_patterns = []
    buy_patterns = []
    for idx, day in df.groupby(df.index.date):
        count_day = day.reset_index()
        for n, bar in enumerate(day.index):
            

           

            if day["Close"].loc[bar] >= day["+band"].loc[bar]:
                state = 1
                u = n*1
                
            if day["Close"].loc[bar] <= day["-band"].loc[bar]:
                state = -1
                d = n*1

            if day["Close"].loc[bar] < day["+band"].loc[bar] and state == 1:
                i = n*1
                state = 2

            if day["Close"].loc[bar] > day["-band"].loc[bar] and state == -1:
                i = n*1
                state = -2

            if day["Close"].loc[bar] < day["WVAP"].loc[bar] and state == 2:
                #if i - p_size >= 30 :

                arr = count_day[["Open","High","Low","Close","Volume"]].iloc[0 : i]

                        
                        
                        #sell_patterns.append(normalize_columns(calculate_differences(arr[["High","Low","Close", "Volume"]].values)))
                        #sell_patterns.append(calculate_differences(arr[["High","Low","Close", "Volume"]].values))
                if arr.shape[0] >= 5:
                          _, H = nmf_decompose(normalize_columns(calculate_differences(arr.values)), 10)
                          sell_patterns.append(H.T)

                    
                        #sell_patterns = sell_patterns.append(pd.DataFrame([{"Close": np.nan, "Volume":np.nan, "WVAP":np.nan, "+band":np.nan, "-band":np.nan}]),ignore_index=True)
                state = 0
                i = 0
                u = 0


            if day["Close"].loc[bar] > day["WVAP"].loc[bar] and state == -2:
                #if i - p_size >= 30 :

                arr = count_day[["Open","High","Low","Close","Volume"]].iloc[ 0 : i]
                        
                        
                        #buy_patterns.append(normalize_columns(calculate_differences(arr[["High","Low","Close", "Volume"]].values)))
                        #buy_patterns.append(calculate_differences(arr[["High","Low","Close", "Volume"]].values))
                if arr.shape[0] >= 5:
                          _ , H = nmf_decompose(normalize_columns(calculate_differences(arr.values)), 10)
                          buy_patterns.append(H.T)

                        #buy_patterns = buy_patterns.append(pd.DataFrame([{"Close": np.nan, "Volume":np.nan, "WVAP":np.nan, "+band":np.nan, "-band":np.nan}]), ignore_index=True)
                state = 0
                i = 0
                d = 0
            

    
    return np.array(sell_patterns), np.array(buy_patterns)



In [6]:
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist
from scipy.sparse import csr_matrix

class DBA:
    def __init__(self, n_iter=10):
        self.n_iter = n_iter
        self.X_mean_ = None

    def fit(self, X):
        self.X_mean_ = np.mean(X, axis=0)
        
        for i in range(self.n_iter):
            dtw_distances = []
            for x in X:
                d, _ = fastdtw(x, self.X_mean_, dist=euclidean)
                dtw_distances.append(d)
            dtw_distances = np.asarray(dtw_distances)
            self.X_mean_ = np.average(X, axis=0, weights=dtw_distances)

    def predict(self, x):
        d, _, = fastdtw(x, self.X_mean_, dist=euclidean)
        return d

In [23]:
def train_neuro(train_df, pattern_join_size, pattern_size):

    sell_patterns, buy_patterns = get_patterns(train_df, pattern_join_size)
    sell_dba = DBA(n_iter=50)

    print(sell_patterns.shape)
    buy_dba =  DBA(n_iter=50)
    sell_dba.fit(sell_patterns)
    buy_dba.fit(buy_patterns)

    
    return sell_dba, buy_dba


def rbm_predict(arr, sell_dba, buy_dba, Signal, treshold):
    arr["Datetime"] = pd.to_datetime(arr["Datetime"])
    arr.set_index("Datetime", inplace=True)
    start_trade = pd.to_datetime("08:00:00")
    end_trade = pd.to_datetime("17:00:00")
    time_loss = 0

    if Signal !=0:
      time_loss+=1
    else:
      time_loss = 0

      


    if arr.index[-1].time() > start_trade.time():
        


        

        if arr["Close"].iloc[-1] >= arr["+band"].iloc[-1]  and arr.index[-1].time() < end_trade.time():
          #dist = sell_dba.predict(normalize_columns(calculate_differences(arr[["High","Low","Close", "Volume"]].values)))
          #dist = sell_dba.predict(calculate_differences(arr[["High","Low","Close", "Volume"]].values))
          _, H = nmf_decompose(normalize_columns(calculate_differences(arr[["Open","High","Low","Close","Volume"]].values)), 10)
          dist = sell_dba.predict(normalize_columns(H.T))
          
         
          if dist < treshold:
            Signal = -1

        if arr["Close"].iloc[-1] <= arr["-band"].iloc[-1]  and arr.index[-1].time() < end_trade.time():
          #dist = buy_dba.predict(normalize_columns(calculate_differences(arr[["High","Low","Close", "Volume"]].values)))
          #dist = buy_dba.predict(calculate_differences(arr[["High","Low","Close", "Volume"]].values))

          _, H = nmf_decompose(normalize_columns(calculate_differences(arr[["Open","High","Low","Close","Volume"]].values)), 10)
          dist = buy_dba.predict(H.T)
          
          if dist < treshold:
            Signal = 1

        if Signal == -1 and arr["Close"].iloc[-1] < arr["WVAP"].iloc[-1]:
            Signal = 0

        if Signal == 1 and arr["Close"].iloc[-1] > arr["WVAP"].iloc[-1]:
            Signal = 0

        if arr["+band"].iloc[-1] == arr["-band"].iloc[-1]:
            Signal = 0

    else:
        Signal = 0


    if time_loss > 120:
      Signal = 0


    return Signal

In [24]:
df = pd.read_csv("/content/drive/MyDrive/working_dir/AAPL_1minute.csv")
train_df = df.iloc[: 100000]
test_df = df.iloc[100000:120000]

In [25]:
tresh =7.0
pattern_joing_size = 50
pattern_size = 120
sell_patterns, buy_patterns = train_neuro(train_df, pattern_joing_size, pattern_size)
#sell_patterns = remove_popular(sell_patterns, pattern_size,5)
#buy_patterns = remove_popular(buy_patterns, pattern_size,60)

<ipython-input-5-cd4595849a49>:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning:

Maximum number of iterations 200 reached. Increase it to improve convergence.

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning:

Maximum number of iterations 200 reached. Increase it to improve convergence.

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning:

Maximum number of iterations 200 reached. Increase it to improve convergence.

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning:

Maximum number of iterations 200 reached. Increase it 

(491, 5, 10)


In [26]:
test_list =[]
for i in range(test_df.shape[0]):
  if i - pattern_size >=0:
    test_list.append(test_df.iloc[i-pattern_size:i])


In [27]:
Signal = 0
Signals = []
for arr in test_list:
  Signal = rbm_predict(arr,sell_patterns, buy_patterns, Signal, tresh)
  Signals.append(Signal)


Streaming output truncated to the last 5000 lines.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning:

The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning:

Maximum number of iterations 200 reached. Increase it to improve convergence.

<ipython-input-23-c6fd80fd17a2>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
test_df = test_df.iloc[pattern_size:]
test_df["Signal"] = Signals

<ipython-input-28-5630b0c4ed9c>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
test_df = test_df.reset_index(drop=True)

In [30]:
test_df["Datetime"] = pd.to_datetime(test_df["Datetime"])
test_df.set_index("Datetime", inplace=True)

In [31]:
from plotly.subplots import make_subplots
#fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=test_df.index, y=test_df["Close"],
                    mode='lines',
                    name='close'))

'''fig.add_trace(go.Scatter(x=test_df.index, y=test_df["ema12"],
                    mode='lines+markers',
                    name='ema12'))

fig.add_trace(go.Scatter(x=test_df.index, y=test_df["ema26"],
                    mode='lines+markers',
                    name='ema26'))'''

'''fig.add_trace(go.Scatter(x=test_df.index, y=test_df["signal"],
                    mode='lines',
                    name='signal'),secondary_y=True)'''


fig.add_trace(go.Bar(
    x=test_df.index[test_df['Signal'] == -1].tolist(),
    y=[test_df['Close'].max() for i in range(len(test_df.index[test_df['Signal'] == -1].tolist()))],
    name='down',
    marker_color='red', opacity=1, 
))
fig.add_trace(go.Bar(
    x=test_df.index[test_df['Signal'] == 1].tolist(),
    y=[test_df['Close'].max() for i in range(len(test_df.index[test_df['Signal'] == 1].tolist()))],
    name='up',
    marker_color='green',opacity=1, 
))

fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df['WVAP'],
    mode='lines',
    name='VWAP', 
    line=dict(color='coral',width=2)
))
fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df["+band"],
    mode='lines',
    name='VWAP + 1std', 
    line=dict(color='green',width=2)
))

fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df["-band"],
    mode='lines',
    name='VWAP - 1std', 
    line=dict(color='green',width=2)

))

'''fig.add_trace(go.Bar(
    x=df.index[df['Signal'] == 0].tolist(),
    y=[df['Close'].max() for i in range(len(df.index[df['Signal'] == 0].tolist()))],
    name='cash',
    marker_color='blue',opacity=0.5, 
))'''

'''fig.update_xaxes(title_text='Index')
fig.update_yaxes(title_text='Close')
for i in range(0,len(df2),440):
  fig.add_vrect(x0=i, x1=i+1, annotation_text="forward end")'''


#fig.add_vrect(x0=1760, x1=1761, annotation_text="forward end")'''
fig.update_xaxes(type='category')
fig.update_layout(yaxis=dict(range=[test_df['Close'].min()-1,test_df['Close'].max()+1]))
fig.update_layout(title_text='Antropos.Разметка на тесте c использованием индикатора MACD. EFA timeframe 1m')
fig.show()







In [ ]:
test_df[["Datetime","Open","High","Low","Close","Volume","Signal"]].to_csv('Atropos_MACD.csv')